In [ ]:
# Required imports
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils.general import non_max_suppression
from utils.augmentations import letterbox
from utils.torch_utils import select_device 

# Load the device (using CPU in this case). Change to 'cuda' for GPU if available.
  
device = select_device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the trained model
model = torch.load('yolov5s.pt', map_location=device)['model'].float()
model.eval()  # Set to evaluation mode

# Inference function
def run_inference(image_path):
    img = cv2.imread(image_path)  # Read image from the given path 
    if img is None:
        print(f"Error: Image not found at {image_path}")
        return None
    
    # Resize image using letterbox
    img_resized = letterbox(img, new_shape=640)[0] 
    resized_height, resized_width = img_resized.shape[:2]
    original_height, original_width = img.shape[:2]

    # Prepare the image for model inference
    img_resized = np.transpose(img_resized, (2, 0, 1))  # Change dimension order
    img_resized = np.expand_dims(img_resized, axis=0)  # Add batch dimension
    img_resized = np.asarray(img_resized) / 255.0  # Normalize image

    # Run inference
    with torch.no_grad():
        pred = model(torch.from_numpy(img_resized).float().to(device))[0]
        pred = non_max_suppression(pred, 0.4, 0.5)[0]  # Apply NMS

    return pred, original_width, original_height, resized_width, resized_height

# Display results function
def display_results(image_path, predictions, original_width, original_height, resized_width, resized_height):
    img = cv2.imread(image_path)
    if predictions is None:
        print("No predictions to display.")
        return
    
    for pred in predictions:
        pred = pred.cpu().numpy()  # Convert tensor to NumPy array
        x_min, y_min, x_max, y_max, conf, cls = pred

        # Calculate scale factors
        scale_x = original_width / resized_width
        scale_y = original_height / resized_height

        # Rescale coordinates to original image size
        x_min, y_min, x_max, y_max = (
            int(x_min * scale_x), 
            int(y_min * scale_y), 
            int(x_max * scale_x), 
            int(y_max * scale_y)
        )

        # Apply confidence threshold
        if conf >= 0.45:
            print(f"Bounding Box: ({x_min}, {y_min}), ({x_max}, {y_max}) - Conf: {conf:.2f}, Class: {int(cls)}")
            label = f"Class {int(cls)} Conf {conf:.2f}"
            color = (0, 255, 0)  # Green for boxes
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)
            cv2.putText(img, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("YOLOv5 Inference Results", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Run inference
image_path = '/Users/sileshi/Desktop/data/sample.jpg'
predictions, original_width, original_height, resized_width, resized_height = run_inference(image_path)

# Display the results
if predictions is not None:
    display_results(image_path, predictions, original_width, original_height, resized_width, resized_height)
else:
    print("No detections found.")
